In [1]:
%reload_ext autoreload
%autoreload 2

setup_already_done = False

In [2]:
import sys
import os

In [3]:
IN_COLAB = 'google.colab' in sys.modules
IN_COLAB

False

In [4]:
if not setup_already_done:
    if(IN_COLAB):
        !git clone https://github.com/Silver0x10/VideoPrediction_MovingMNIST.git
        %cd VideoPrediction_MovingMNIST
    else:
        %cd ..
    if(not os.path.exists("data/mnist_test_seq.npy")):
        if(not os.path.exists("data")): 
            %mkdir data
        %cd data
        !wget -q https://www.cs.toronto.edu/~nitish/unsupervised_video/mnist_test_seq.npy
        %cd ..
        
    # %pip install -qr requirements.txt  --quiet
    setup_already_done = True

/home/silverio/Projects/uni/DeepLearning/VideoPrediction_MovingMNIST


In [20]:
# import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader,random_split
# from torch import optim, nn, utils, Tensor
# import torchvision
# from torchvision.transforms import ToTensor
import lightning.pytorch as pl

In [6]:
# !nvidia-smi

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [28]:
from src.MovingMNIST import MovingMNIST

dataset = MovingMNIST(data_path="data/mnist_test_seq.npy")
train_set, validation_set, test_set = random_split(dataset,[8000, 1000, 1000],
                                                   generator=torch.Generator().manual_seed(42))
# Visualization Test
dataset.visualize(0)

In [30]:
loader = DataLoader(dataset)
training_dataloader = DataLoader(train_set)
validation_dataloader = DataLoader(validation_set)
test_dataloader = DataLoader(test_set)

In [31]:
from src.simpleLSTM import simpleLSTM

model = simpleLSTM()


In [32]:
# for data in loader:
#     print(model.training_step(data,0))
#     break

In [12]:
# from lightning.pytorch.loggers.wandb import WandbLogger
# logger = WandbLogger()

In [35]:
# trainer = pl.Trainer(limit_train_batches=1, max_epochs=1, accelerator="cpu")
# trainer = pl.Trainer(max_epochs=1, limit_train_batches=100, accelerator="cpu")
# trainer = pl.Trainer(max_epochs=5, limit_train_batches=100)
trainer = pl.Trainer(max_epochs=5, accelerator="gpu")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [33]:
trainer.fit(model=model, train_dataloaders=training_dataloader, val_dataloaders=validation_dataloader)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/silverio/.local/lib/python3.10/site-packages/lightning/pytorch/trainer/setup.py:176: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
Missing logger folder: /home/silverio/Projects/uni/DeepLearning/VideoPrediction_MovingMNIST/lightning_logs

  | Name    | Type       | Params
---------------------------------------
0 | loss_fn | MSELoss    | 0     
1 | relu    | ReLU       | 0     
2 | encoder | Sequential | 4.2 M 
3 | lstm    | LSTM       | 3.1 M 
4 | decoder | Sequential | 4.7 M 
---------------------------------------
12.1 M    Trainable params
0         Non-trainable params
12.1 M    Total params
48.275    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/silverio/.local/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/silverio/.local/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


In [34]:
trainer.test(model, dataloaders=test_dataloader)

/home/silverio/.local/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, test_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │     2687.852783203125     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 2687.852783203125}]

In [19]:
# !zip -r ../logs lightning_logs/ 

tensor(2802.6553, grad_fn=<DivBackward0>)

In [15]:
%reload_ext tensorboard
%tensorboard --logdir=lightning_logs/ #--port 6007

In [16]:
# import gc
# print(torch.cuda.list_gpu_processes())
# gc.collect()
# torch.cuda.empty_cache()